In [1]:
from code.excelmem.excelmemvis import main as excel_main
from code.libremem.librememvis import main as libre_main

import plotly.graph_objects as go
import multiprocessing as mp

In [2]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

FigureWidget({
    'data': [{'name': 'Formula-Value (MB)', 'type': 'scatter', 'uid': 'a075bf75-37f0-4571-9fdd-…

In [3]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'scatter', 'uid': '020e4219-b76f-4347-a924-559…

In [4]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'bar', 'uid': 'd3f7dba9-0bbd-4ad0-8e6f-e42865c…

In [5]:
# Either "libre" or "excel"
APPLICATION = "excel"

SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
INPUTS_PATH = os.path.join("input-data", "tests")
FV_INPUTDIR = "formula-value"
VO_INPUTDIR = "value-only"
OUTPUT_NAME = "results"
OUTDIR_NAME = "test"
INTEGER_ARG = 3

In [6]:
parent_conn, child_conn = mp.Pipe()

if APPLICATION.lower() == "excel":
  process = mp.Process(target=excel_main
    , args=(child_conn, INPUTS_PATH,)
    , kwargs={
      "fv_inputdir"   : "formula-value"
      , "vo_inputdir" : "value-only"
      , "output_path" : os.path.join(OUTPUT_NAME, "excel")
      , "outdir_name" : OUTDIR_NAME
      , "trials"      : INTEGER_ARG
    }
  )
else:
  process = mp.Process(target=libre_main
    , args=(child_conn, INPUTS_PATH,)
    , kwargs={
      "soffice_path"  : SOFFICE_DIR
      , "fv_inputdir" : FV_INPUTDIR
      , "vo_inputdir" : VO_INPUTDIR
      , "output_path" : os.path.join(OUTPUT_NAME, "libre")
      , "outdir_name" : OUTDIR_NAME
      , "pollseconds" : INTEGER_ARG
    }
  )

In [7]:
process.start()
while True:
    item = parent_conn.recv()
    if type(item) == dict:
        vo_rowsizes = []; vo_memsizes = []; fv_rowsizes = []; fv_memsizes = []
        for r, d in sorted(item.items(), key=lambda pair: pair[0]):
            if "Value USS (MB)"   in d: vo_rowsizes.append(str(r)); vo_memsizes.append(d["Value USS (MB)"  ])
            if "Formula USS (MB)" in d: fv_rowsizes.append(str(r)); fv_memsizes.append(d["Formula USS (MB)"])
        barplt.data[0].x = vo_fig.data[0].x = vo_rowsizes
        barplt.data[1].x = fv_fig.data[0].x = fv_rowsizes
        barplt.data[0].y = vo_fig.data[0].y = vo_memsizes
        barplt.data[1].y = fv_fig.data[0].y = fv_memsizes
    if type(item) == str:   print(item)
    if item is None:        break
process.join()
process.terminate()

Closing all running instances of EXCEL.EXE (if any)
Opening vo-10000.xlsx (trial 1)
Opening vo-10000.xlsx (trial 2)
Opening vo-10000.xlsx (trial 3)
{'Value Peak WSS (MB)': 141.77484799999993, 'Value WSS (MB)': 139.30086399999993, 'Value RSS (MB)': 139.300864, 'Value USS (MB)': 60.645375999999985}
Opening vo-100.xlsx (trial 1)
Opening vo-100.xlsx (trial 2)
Opening vo-100.xlsx (trial 3)
{'Value Peak WSS (MB)': 138.84620799999996, 'Value WSS (MB)': 138.84211200000001, 'Value RSS (MB)': 138.84211200000001, 'Value USS (MB)': 59.916288000000016}
Opening vo-200000.xlsx (trial 1)
Opening vo-200000.xlsx (trial 2)
Opening vo-200000.xlsx (trial 3)
{'Value Peak WSS (MB)': 150.757376, 'Value WSS (MB)': 150.75328, 'Value RSS (MB)': 150.75328, 'Value USS (MB)': 71.73734399999996}
Opening vo-10.xlsx (trial 1)
Opening vo-10.xlsx (trial 2)
Opening vo-10.xlsx (trial 3)
{'Value Peak WSS (MB)': 138.23999999999998, 'Value WSS (MB)': 138.22771199999997, 'Value RSS (MB)': 138.22771199999997, 'Value USS (MB)':

In [8]:
# EXCEL:
# RCBS-1col
#   Up to 1 million: Total time (HH:MM:SS): 06:56:18
# RLS-1col
#   Up to 1 million: Total time (HH:MM:SS): 00:06:18
# RLS-5col
#   Up to 1 million: Total time (HH:MM:SS): 00:21:21